In [1]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os
import numpy as np

### RESNET 32

In [2]:
# Sagemaker Notebook must be of type, conda_pytorch_p36
!pip install nvidia-ml-py3
!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19189 sha256=ff409f1c7762a56588ee1daa14a38c86ffee112f67f64379896d157212f0db64
  Stored in directory: /home/ec2-user/.cache/pip/wheels/7f/26/a3/33f2079871e2bebb3f53a2b21c3ec64129b8efdd18a6263a52
Successfully built nvidia-ml-py3
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
  Cloning https://github.com/NVIDIA/apex.git to /tmp/pip-install-8bnxf0m1/apex
  Running command git clone -q https://github.com/NVIDIA/apex.git /tmp/pip-install-8bnxf0m1/apex
  Running command git submodule update --init --recursive -q
  Created wheel for fvcore: filename=fvcore-0.1.1.post20200704-py3-none-any.whl size=41891 sha256=b0bded80cf4c78b19a7a6ed74f249823ec260af26d4d335930b594c190b80ed5
  Stored in directory: /home/ec2-user/.cache/pip/wheels/fc/e7/9a/a35290ff6bb619b7c14477a31639990d58544bef99de6f7227
  Created wheel for apex: filename=ape

In [3]:
# Need to add this to requirements.txt
!pip install tensorboard

     |████████████████████████████████| 3.0 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 39.6 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 34.3 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 42.2 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 55.1 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 50.8 MB/s eta 0:00:01
  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none-any.whl size=121931 sha256=bcd65861e967a0a92a266d84166d02a7e7ebef53cc123122f25755df1ccba6d8
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/af/84/3962a6af7b4ab336e951b7877dcfb758cf94548bb1771e0679
Successfully built absl-py
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [67]:
#Train the model per the settings specified in the original RESNET paper
# os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
# !python train.py --config configs/cifar/resnet.yaml \
#     model.resnet.depth 32 \
#     train.batch_size 128 \
#     dataset.name CIFAR10 \
#     train.base_lr 0.1 \
#     train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00 \
#     scheduler.epochs 1
pass

In [13]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/resnet.yaml \
    model.resnet.depth 32 \
    train.batch_size 128 \
    dataset.name CIFAR10_RA_1_20 \
    train.base_lr 0.1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00 \
    scheduler.epochs 400

[2020-07-10 19:47:56] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_RA_1_20
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 32
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck
 

In [20]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
#!python train.py --config configs/cifar/resnet.yaml \
!python train.py --config /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/checkpoint_00400.pth \
    dataset.name CIFAR10 \
    train.base_lr .001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00_resume400_50 \
    scheduler.epochs 50

#### Set LEARNING RATE based on ending LR


Traceback (most recent call last):
  File "train.py", line 436, in <module>
    main()
  File "train.py", line 336, in main
    f'Output directory `{output_dir.as_posix()}` already exists')
RuntimeError: Output directory `/home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/exp00_resume400_50` already exists


In [39]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset - without training on unaugmented data
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/test_results_0400_cifar10

[2020-06-21 21:43:10] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:02<00:00, 26.40it/s]
[2020-06-21 21:43:14] __main__ INFO: Elapsed 3.00
[2020-06-21 21:43:14] __main__ INFO: Loss 1.7503 Accuracy 0.7241


In [29]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00_resume400_50/test_results_0050_cifar10

python: can't open file 'evaluate.py': [Errno 2] No such file or directory


In [30]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset - without training on unaugmented data
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/test_results_0400_cifar101

[2020-07-03 19:29:47] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:03<00:00,  4.97it/s]
[2020-07-03 19:29:50] __main__ INFO: Elapsed 3.22
[2020-07-03 19:29:50] __main__ INFO: Loss 3.2528 Accuracy 0.5230


In [31]:
#npzf = np.load('resnet_basic_32_ra_2_20/predictions_400_CIFAR101.npz')
npzf = np.load('/home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/test_results_0400_cifar101/predictions.npz')
#experiments/resnet_basic_32_ra_2_20/exp00/test_results_0400_cifar101/predictions.npz
print(len(npzf['labels']))

2000


In [14]:
!pwd

/home/ec2-user/SageMaker/w210-capstone/aws-notebooks


In [44]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00_resume400_50/test_results_0050_cifar101

[2020-06-21 21:47:35] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_20/exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 17.36it/s]
[2020-06-21 21:47:37] __main__ INFO: Elapsed 0.92
[2020-06-21 21:47:37] __main__ INFO: Loss 0.7639 Accuracy 0.7685


In [32]:
import pandas as pd
a = pd.Series(['resnet_basic_32_ra_1_20', 400, 'cifar10', 1.7503, 0.7241]) #Loss 1.7503 Accuracy 0.7241
b = pd.Series(['resnet_basic_32_ra_1_20', 400, 'cifar10.1', 3.2528, 0.5230]) #Loss 3.2528 Accuracy 0.5230

c = pd.Series(['resnet_basic_32_ra_1_20_refined400', 50, 'cifar10', 0.4279, 0.8670]) #Loss 0.4279 Accuracy 0.8670
d = pd.Series(['resnet_basic_32_ra_1_20_refined400', 50, 'cifar10.1', 0.8061, 0.7645]) #Loss 0.8061 Accuracy 0.7645
               
df_results = pd.concat([a,b,c,d], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 92.5 if row[2] == 'cifar10' else 84.9), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (92.0, 93.0) if row[2] == 'cifar10' else (83.2, 86.4)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,resnet_basic_32_ra_2_20,400,cifar10,1.7503,0.7241,92.5,"(92.0, 93.0)"
1,resnet_basic_32_ra_2_20,300,cifar10,1.6118,0.7259,92.5,"(92.0, 93.0)"
2,resnet_basic_32_ra_2_20,400,cifar10.1,3.2528,0.523,84.9,"(83.2, 86.4)"
3,resnet_basic_32_ra_2_20,300,cifar10.1,3.047,0.5265,84.9,"(83.2, 86.4)"
4,resnet_basic_32_ra_2_20_refined400,50,cifar10,0.4279,0.867,92.5,"(92.0, 93.0)"
5,resnet_basic_32_ra_2_20_refined300,150,cifar10,0.4342,0.874,92.5,"(92.0, 93.0)"
6,resnet_basic_32_ra_2_20_refined400,50,cifar10.1,0.8061,0.7645,84.9,"(83.2, 86.4)"
7,resnet_basic_32_ra_2_20_refined300,150,cifar10.1,0.7919,0.7635,84.9,"(83.2, 86.4)"


In [35]:
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/test_results_0400_cifar101/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/test_results_0400_cifar101/predictions.npz'

In [9]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-may29'
prefix = 'sagemaker/results/original-models/resnet_basic_32'
path = '/home/ec2-user/SageMaker/experiments/resnet_basic_32'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)